## setup

In [1]:
import Interface as I

/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/model_data_base/distributed_lock.py:13: UserWarning: environment variable ISF_DISTRIBUTED_LOCK_CONFIG is not set. Falling back to default configuration.
  warnings.warn('environment variable ISF_DISTRIBUTED_LOCK_CONFIG is not set. ' +
/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/model_data_base/distributed_lock.py:33: UserWarning: Using file based locking.Please be careful on nfs mounts as file based locking has issues in this case.
  warnings.warn('Using file based locking.'


trying to connect to distributed locking server {'type': 'redis', 'config': {'host': 'spock', 'port': 8885, 'socket_timeout': 1}}
trying to connect to distributed locking server {'type': 'redis', 'config': {'host': 'localhost', 'port': 6379, 'socket_timeout': 1}}
trying to connect to distributed locking server {'type': 'file'}
Current version: heads/vcbm-2023+0.ged52ae60.dirty
Current pid: 996570


/scratch/visualn/bzfharth/in-silico-install-dir/anaconda_isf3/lib/python3.8/site-packages/pandas_msgpack-0.1.4+14.gfcb0471-py3.8-linux-x86_64.egg/pandas_msgpack/packers.py:78: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Timestamp, Period, Series, DataFrame,  # noqa


Loading mechanisms:


/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/visualize/cell_morphology_visualizer.py:29: ImportWarning: Dash could not be imported. Interactive visualisations will not work.
  warnings.warn(
/scratch/visualn/bzfharth/in-silico-install-dir/project_src/in_silico_framework/Interface.py:190: UserWarning:

The source folder has uncommited changes!



setting up local multiprocessing framework ... done


Loaded modules with __version__ attribute are:
IPython: 7.19.0, Interface: heads/vcbm-2023+0.ged52ae60.dirty, PIL: 8.2.0, _csv: 1.0, _ctypes: 1.1.0, _curses: b'2.2', _decimal: 1.70, argparse: 1.1, backcall: 0.2.0, blosc: 1.10.2, bluepyopt: 1.9.126, bottleneck: 1.3.2, cffi: 1.14.3, click: 7.1.2, cloudpickle: 1.6.0, colorama: 0.4.4, csv: 1.0, ctypes: 1.1.0, cycler: 0.10.0, cytoolz: 0.11.0, dask: 2.30.0, dateutil: 2.8.1, deap: 1.3, decimal: 1.70, decorator: 4.4.2, distributed: 2.30.1, distutils: 3.8.5, fsspec: 0.8.3, future: 0.18.2, ipykernel: 5.3.4, ipython_genutils: 0.2.0, ipywidgets: 7.5.1, jedi: 0.17.1, jinja2: 2.11.2, joblib: 0.17.0, json: 2.0.9, jupyter_client: 6.1.7, jupyter_core: 4.6.3, kiwisolver: 1.3.0, logging: 0.5.1.2, markupsafe: 1.1.1, matplotlib: 3.3.2, mkl: 2.3.0, neuron: 7.8.2, numexpr: 2.7.1, numpy: 1.19.2, pandas: 1.1.3, pandas_msgpack: 0.1.4+14.gfcb0471.dirty, parameters: 0.2.1, parso: 0.7.0, past: 0.18.2, pexpect: 

In [2]:
postfix = "_2023-06-22"
mdb = I.ModelDataBase('/scratch/visualn/bzfharth/simulation_data'+postfix)
print(mdb.keys())

['89', '90', 'WR69', '88', '93', '84', '87', 'WR64', '85', '86', '91', 'WR71', 'examplary_models', 'df_all', 'run_gillis_protocol_v1', 'all_models_py3', 'run_gillis_protocol_v1__all_models_genetic_py3', 'all_models_genetic_py3', 'all_models_genetic_py2_1', 'all_models_genetic_py2_2', 'all_models_RW_BAC2step_seed_11_filtered', 'all_models_RW_BAC2step_seed_17_filtered', 'all_models_RW_BAC2step_seed_1_filtered']


In [3]:
import os
import shutil

def makeCleanDir(dirname):
    if(os.path.exists(dirname)):
        shutil.rmtree(dirname, ignore_errors=False, onerror=None)
    os.mkdir(dirname)

selected_dataset = 'all_models_RW_BAC2step_seed_1_filtered'
csv_outfolder = os.path.join('/scratch/visualn/bzfharth/simulation_data_postprocessed', selected_dataset+postfix)

makeCleanDir(csv_outfolder)

In [4]:
# init
import pickle
import biophysics_fitting.hay_evaluation
import project_specific_ipynb_code.biophysical_models
from project_specific_ipynb_code.biophysical_models import get_model_ddf_from_encapsulating_mdb, objectives_BAC
from project_specific_ipynb_code.biophysical_models import get_ddf_selected
from project_specific_ipynb_code.hot_zone import get_cell_object_from_hoc
from project_specific_ipynb_code.biophysical_models import min_max_plot, normalize_params
from project_specific_ipynb_code.biophysical_models import CurrentAnalysis
from project_specific_ipynb_code.biophysical_models import modify_simulator_to_record_apical_dendrite_conductances

In [5]:
def clean_df(df, max_nans=3000):
    blocked_columns = []
    columns_many_nan = df.columns[df.isna().sum() > max_nans].to_list()
    boolean_columns = df.select_dtypes(include=bool).columns.to_list()
    object_columns = df.select_dtypes(include=object).columns.to_list()
    
    columns_to_drop = blocked_columns + columns_many_nan + boolean_columns + object_columns

    df_pruned_cols = df.drop(columns_to_drop, axis=1)
    df_cleaned = df_pruned_cols.dropna()
    return df_cleaned



In [11]:
columns_many_nan = df.columns[df.isna().sum() > 3000].to_list()
columns_many_nan

['bAP.check_max_prestim_dendrite_depo',
 '1BAC.check_max_prestim_dendrite_depo',
 '2BAC_ISI.raw',
 '2BAC_ISI.normalized',
 '2BAC_ahpdepth.normalized',
 '2BAC_caSpike_height.check_ca_max_after_nth_somatic_spike',
 '2BAC.check_max_prestim_dendrite_depo',
 'mf1',
 'AI1',
 'ISIcv1',
 'DI1',
 'TTFS1',
 'APh1',
 'fAHPd1',
 'sAHPd1',
 'sAHPt1',
 'APw1',
 'mf2',
 'AI2',
 'ISIcv2',
 'DI2',
 'TTFS2',
 'APh2',
 'fAHPd2',
 'sAHPd2',
 'sAHPt2',
 'APw2',
 'mf3',
 'AI3',
 'ISIcv3',
 'DI3',
 'TTFS3',
 'APh3',
 'fAHPd3',
 'sAHPd3',
 'sAHPt3',
 'APw3',
 'constants_bifurcation.NaTa_t.gNaTa_tbar',
 'constants_bifurcation.Ca_LVAst.gCa_LVAstbar',
 'constants_bifurcation.Ca_HVA.gCa_HVAbar',
 'constants_bifurcation.Ih.gIhbar',
 'constants_bifurcation.Im.gImbar',
 'constants_bifurcation.SKv3_1.gSKv3_1bar',
 'constants_bifurcation.SK_E2.gSK_E2bar',
 'constants_bifurcation.ek',
 'constants_bifurcation.ena',
 'constants_prox.NaTa_t.gNaTa_tbar',
 'constants_prox.Ca_LVAst.gCa_LVAstbar',
 'constants_prox.Ca_HVA.gCa_

In [6]:
folder = mdb[selected_dataset]
folder

'/scratch/visualn/bzfharth/simulation_data_2023-06-22/all_models_rw_bac2step_seed_1_filtered_rqv6y_nj_'

In [7]:
import pandas_msgpack
import glob

files = glob.glob(os.path.join(folder, "pandas_to_msgpack_*"))
files_dict = {}
for file in files:
    idx = int(file.split("_")[-1])
    files_dict[idx] = file

file_indices = sorted(files_dict.keys())